<a href="https://colab.research.google.com/github/juanpaat/Machine-Learning-Project-Template/blob/main/Template_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Load/Create the data

## Creating a Dataset

In [ ]:
# Creating a manual dataset
data = pd.DataFrame({
    'name': ['John', 'Jane', 'Jack', 'John', None],
    'age': [28, 34, None, 28, 22],
    'purchase_amount': [100.5, None, 85.3, 100.5, 50.0],
    'date_of_purchase': ['2023/12/01', '2023/12/02', '2023/12/01', '2023/12/01', '2023/12/03']
	})



## Read CVS Files

In [ ]:
import pandas as pd

df = pd.read_csv(
    "data/sheffield_weather_station.csv",  # Replace with your CSV file path
    # The following arguments are optional and can be removed:
    # If columns aren't separated by commas, indicate the delimiter here
    sep="\s+",
    # Indicate which zero-indexed row number(s) have the column names
    header=0,
    # List of column names to use (useful for renaming columns)
    names=["year", "month", "max_c", "min_c", "af", "rain", "sun"],
    # If not all columns are needed, indicate which you need (useful for lower memory usage)
    usecols=["year", "month", "max_c", "min_c", "rain", "sun"],
    # Indicate which column(s) to use as row labels
    index_col=["year", "month"],
    # Lines starting with this string should be ignored (useful if there are file comments)
    comment="#",
    # Indicate the number of lines to skip at the start of the file (also useful for file comments)
    skiprows=None,
    # Indicate string(s) that should be recognized as NaN/NA
    na_values=["---", "unknown", "no info"],
    # Indicate which column(s) are date column(s)
    parse_dates=False,
    # Indicate number of rows to read (useful for large files)
    nrows=500,
    # Encoding to use when reading file
    encoding="utf-8",
)

## Read Excel Files

In [ ]:
import pandas as pd

df = pd.read_excel(
    "data/employee_information.xlsx",  # Replace with your Excel file path
    # The following arguments are optional and can be removed:
    # By default pandas will only read the first sheet
    # You can change that by specifying the sheet name(s) or zero-indexed sheet position(s)
    sheet_name="Employee Addresses",
    # Indicate which zero-indexed row number(s) have the column names
    header=1,
    # If not all columns are needed, indicate which you need (useful for lower memory usage)
    usecols=(0, 1, 3, 4, 5, 6),
    # List of column names to use (useful for renaming columns)
    names=["id", "lastname", "country", "city", "street", "number"],
    # Indicate which column(s) to use as row labels
    index_col="id",
    # Lines starting with this string should be ignored (useful if there are file comments)
    comment="Last updated:",
    # Indicate the number of lines to skip at the start of the file (also useful for file comments)
    skiprows=1,
    # Indicate string(s) that should be recognized as NaN/NA
    na_values=["---", "unknown", "no info"],
    # Indicate which column(s) are date column(s)
    parse_dates=False,
    # Indicate number of rows to read (useful for large files)
    nrows=500,
)

# Step 2: Understand the data

## Shape of a dataset

In [ ]:
df.shape

## Data description

In [ ]:
X.describe()

## Number of observations per group

In [ ]:
y_train.value_counts()

## Outliers

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats

In [ ]:
import pandas as pd

def extract_outliers_from_boxplot(series: pd.Series):
    """Extract outliers from a Pandas Series based on the IQR method."""

    # Ensure input is a Pandas Series
    if not isinstance(series, pd.Series):
        raise TypeError("Input must be a Pandas Series")

    # Calculate IQR (Interquartile Range)
    iqr_q1 = series.quantile(0.25)
    iqr_q3 = series.quantile(0.75)
    med = series.median()

    # Finding the IQR region
    iqr = iqr_q3 - iqr_q1

    # Finding upper and lower whiskers
    upper_bound = iqr_q3 + (1.5 * iqr)
    lower_bound = iqr_q1 - (1.5 * iqr)

    # Extract outliers
    outliers = series[(series <= lower_bound) | (series >= upper_bound)]

    print(f'Outliers within the box plot are: {outliers.values}')
    return outliers

In [ ]:
def extract_outliers_from_boxplot(array):
    ## Get IQR
    iqr_q1 = np.quantile(array, 0.25)
    iqr_q3 = np.quantile(array, 0.75)
    med = np.median(array)

    # finding the iqr region
    iqr = iqr_q3-iqr_q1

    # finding upper and lower whiskers
    upper_bound = iqr_q3+(1.5*iqr)
    lower_bound = iqr_q1-(1.5*iqr)

    outliers = array[(array <= lower_bound) | (array >= upper_bound)]
    print('Outliers within the box plot are :{}'.format(outliers))
    return outliers

extract_outliers_from_boxplot(df['purchases'])

## Box and violin plots

In [ ]:
import seaborn as sns

plt.violinplot(df['purchases'])
sns.boxplot(df['purchases'])

## Winsorize  

Replace the outlier with a threshold value

In [ ]:
def winsorize(df, column, upper, lower):
    """
    Apply Winsorization to a specified column in a DataFrame.

    Parameters:
    ----------
    df : pandas.DataFrame
        The input DataFrame containing the data.
    column : str
        The name of the column to apply Winsorization on.
    upper : float
        The upper percentile threshold (0-100). Values above this percentile
        will be replaced with the percentile value.
    lower : float
        The lower percentile threshold (0-100). Values below this percentile
        will be replaced with the percentile value.

    Returns:
    -------
    pandas.DataFrame
        The DataFrame with the specified column Winsorized.
    """

    col_df = df[column]

    perc_upper = np.percentile(df[column],upper)
    perc_lower = np.percentile(df[column],lower)

    df[column] = np.where(df[column] >= perc_upper,
                          perc_upper,
                          df[column])

    df[column] = np.where(df[column] <= perc_lower,
                          perc_lower,
                          df[column])

    return df


# Example
winsorize(df, 'purchases', 97.5, 0.025)

## Z-Scores  

Similar method to boxplots but we can specify the percentile we want to use, to classify a point as an outlier.

In [ ]:
import scipy.stats


purchases = df['purchases']

def percentile_outliers(array,
                        lower_bound_perc,
                        upper_bound_perc):

    upper_bound = np.percentile(df['purchases'], upper_bound_perc)
    lower_bound = np.percentile(df['purchases'], lower_bound_perc)

    outliers = array[(array <= lower_bound) | (array >= upper_bound)]

    return outliers

def z_score_outliers(array,
                     z_score_lower,
                     z_score_upper):

    z_scores = scipy.stats.zscore(array)
    outliers = (z_scores > 1.96) | (z_scores < -1.96)

    return array[outliers]



# example
outliers = percentile_outliers(df['purchases'],
               upper_bound_perc = 99,
               lower_bound_perc = 1)


# Example
z_score_outliers(df['purchases'],
                 z_score_lower = -1.96,
                 z_score_upper = 1.96)

## Remove

In [ ]:
def z_score_removal(df, column, lower_z_score, upper_z_score):

    col_df = df[column]

    z_scores = scipy.stats.zscore(purchases)
    outliers = (z_scores > upper_z_score) | (z_scores < lower_z_score)
    return df[~outliers]

def percentile_removal(df, column, lower_bound_perc, upper_bound_perc):

    col_df = df[column]

    upper_bound = np.percentile(col_df, upper_bound_perc)
    lower_bound = np.percentile(col_df, lower_bound_perc)

    z_scores = scipy.stats.zscore(purchases)
    outliers = (z_scores > upper_bound) | (z_scores < lower_bound)
    return df[~outliers]


filtered_df = z_score_removal(df, 'purchases', -1.96, 1.96)
percentile_removal(df, 'purchases', lower_bound_perc = 1, upper_bound_perc = 99)

# Step 3: Plots


## Single Variable Plots


In [ ]:
#import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

### Histograms


In [ ]:
df.variable.hist(bins = 100)

plt.hist(df_agg['variable'])

df_agg['variable'].hist()

### Bar charts

In [ ]:
#get count of number of observations by variable
rev_values = df_agg['variable'].value_counts()
rev_values.plot.bar()

### Box Plots


In [ ]:
plt.boxplot(df_agg['variable'])

## Relationships & Multi-variable plots


### Scatterplots


In [ ]:
#create plot with matplotlib
plt.scatter(df['Average percentage viewed (%)'] ,df['CPM (USD)'])

#let's do the same thing with seaborn so we can see a trendline.
sns.regplot(x='Average percentage viewed (%)', y ='CPM (USD)', data = df)

### Correlation Matrices


In [ ]:
corr = df.corr()

sns.heatmap(corr)

In [ ]:
#A better example (formatting used in below chart) - https://seaborn.pydata.org/examples/many_pairwise_correlations.html

sns.set_theme(style="white")

# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle (otherwise this looks like the square we had above and is redundant)
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 10))

# Generate a custom diverging colormap (choose colors here)
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True, annot_kws={"fontsize":8})

#obviously many of thes variables are HIGHLY correlated. Something we may want to explore is why Average percentage viewed is negatively related to RPM

### Pivot Tables


In [ ]:
#pivot table to explore values

pd.pivot_table(df_agg_country_sub, index = 'Country Code', values = 'Average View Percentage')

# more complex
pd.pivot_table(df_agg_country_sub, index = 'Country Code', columns = 'Is Subscribed',values = 'Average View Percentage')


#create new pivot tables
rm_x_date = pd.pivot_table(df_ts, index='Month_Year',values = 'User Subscriptions Removed', aggfunc ='sum').reset_index()

### Bar Charts


In [ ]:
#Let's plot a slightly simpler graph. Let's just look at if subscribers or non-subsribers watch my videos for longer
pd.pivot_table(df_agg_country_sub, index = 'Is Subscribed', values = 'Average View Percentage').plot.bar()

### Line Charts

In [ ]:
#First we need to make sure our date field is in the date time format. We do this by converting our string to datetime
df['Date'] = pd.to_datetime(df['Date'])

#first, we have to aggregate these by a variable. We do this with a pivot table.
rm_x_date = pd.pivot_table(df, index = 'Date',values = 'User Subscriptions Removed', aggfunc ='sum').reset_index()

#next we visualize this data with seaborn
sns.lineplot(data=rm_x_date,x='Date', y='User Subscriptions Removed')

# One can compare over months instead of days.
df_ts['Month_Year'] = df_ts['Date'].dt.to_period('M')

"""
B: business day frequency
D: calendar day frequency
W: weekly frequency
M: monthly frequency
Q: quarterly frequency
Y: yearly frequency
h: hourly frequency
min: minutely frequency
s: secondly frequency
ms: milliseconds
us: microseconds
ns: nanoseconds
"""

# Step 4: Clean the data

## Rename columns

In [ ]:
# Rename columns for consistency
df.rename(columns={'Cust Name': 'Customer_Name', 'Amt': 'Amount'}, inplace=True)


## Handling missing values

In [ ]:
# Checking Null Values
df.isnull().sum()

In [ ]:
def nulls_summary_table(df):
    """
    Returns a summary table showing null value counts and percentage

    Parameters:
    df (DataFrame): Dataframe to check

    Returns:
    null_values (DataFrame)
    """
    null_values = pd.DataFrame(df.isnull().sum())
    null_values[1] = null_values[0]/len(df)
    null_values.columns = ['null_count','null_pct']
    return null_values

nulls_summary_table(df)

## Mean/Median/Mode Imputation


In [ ]:
# Handling missing values using mean imputation for 'age' and 'purchase_amount'
imputer = SimpleImputer(strategy='mean') # "median" / "median" / "most_frequent" / "constant"
data[['age', 'purchase_amount']] = imputer.fit_transform(data[['age', 'purchase_amount']])


# Fill missing values with a constant 0
df['Amount'].fillna(0, inplace=True)

In [ ]:
## Mean
X_train_m.loc[:,'age'] = X_train_m['age'].fillna(np.mean(X_train_m['age']))
X_test_m.loc[:,'age'] = X_test_m['age'].fillna(np.mean(X_train_m['age'])) ## Cannot use training dataset to impute


X_train_m.loc[:,'days_on_platform'] = X_train_m['days_on_platform'].fillna(np.mean(X_train_m['days_on_platform']))
X_test_m.loc[:,'days_on_platform'] = X_test_m['days_on_platform'].fillna(np.mean(X_train_m['days_on_platform'])) ## Cannot use training dataset to impute

## Median
m_df.loc[:,'age'] = df['age'].fillna(np.median(m_df['age']))

## Mode
m_df.loc[:,'age'] = m_df['age'].fillna(stats.mode(m_df['age'])[0][0])

## Multiple Imputation Using Regression

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

## Target - Purchases in the first six months

r_df = df.copy()

X_r = r_df[['age','days_on_platform','income']]
y_r = r_df['lifetime_value']


X_train_r = X_r[:4000]
y_train_r = y_r[:4000]

X_test_r = X_r[1000:]
y_test_r = y_r[1000:]


Imp = IterativeImputer(max_iter=10, random_state = 0)
Imp.fit(X_train_r)

X_train_r = Imp.transform(X_train_r)
X_test_r = Imp.transform(X_test_r)

X_train_r = pd.DataFrame(X_train_r)
X_train_r.columns = X_train_r.columns

X_test_r = pd.DataFrame(X_test_r)
X_test_r.columns = X_test_r.columns

r_df = pd.concat([X_train_r,X_test_r],axis = 0)

## Nearest Neighbor Imputation


In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5, weights="uniform")
imputer.fit(X_train_r)
X_train_k = imputer.transform(X_train_r)
X_test_k = imputer.transform(X_test_r)

y_train_k = y_train_r.copy()
y_test_k = y_test_r.copy()

## Removing duplicate rows

In [ ]:
# Removing duplicate rows
data = data.drop_duplicates()

#or

# Drop duplicates
df.drop_duplicates(inplace=True)

## Correcting inconsistent date formats

In [ ]:
# Correcting inconsistent date formats
data['date_of_purchase'] = pd.to_datetime(data['date_of_purchase'], errors='coerce')

# Categorical Features

## One Hot Encoding


In [ ]:
#Using pandas

# Specify the columns you wish to one-hot encode
categorical_columns = [
    "job",
    "marital"
]

# Perform the one-hot encoding
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# View the resulting DataFrame
df_encoded

In [ ]:
# another option using Sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Specify the columns you wish to one-hot encode
categorical_columns = ["job", "marital"]

# Filter the DataFrame for the categorical features
cat_features = df[categorical_columns]

# Initialize the OneHotEncoder and fit it to the categorical features
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
enc.fit(cat_features)

# Use the transform method to one hot encode the categorical data and then convert it to a DataFrame
enc_data = pd.DataFrame(
    enc.transform(cat_features),
    columns=enc.get_feature_names_out(categorical_columns)
)

# Join with the rest of the data and preview the DataFrame
df_encoded = df.join(enc_data)
df_encoded

In [ ]:
#  Label Encoder
from sklearn.preprocessing import LabelEncoder

# Create a copy of the original DataFrame
df_encoded = df.copy()

# Specify the column you wish to one-hot encode
label_column = "education"

# Initialize the LabelEncoder
le = LabelEncoder()

# Create a new column using the fit_transform method of the LabelEncoder
df_encoded[label_column + "_enc"] = le.fit_transform(df_encoded[label_column])

# Preview the original and encoded column
df_encoded[[label_column, label_column + "_enc"]]

## Ordinal Encoding


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

## Get the data we want to encode, convert to unique values
data = np.asarray(df[['room_type']])

# define ordinal encoding
encoder = OrdinalEncoder()
# transform data
result = encoder.fit_transform(data)



## Aggregate it back into original dataframe
ord_encoded = pd.DataFrame(result)
ord_encoded.columns = ['room_type_ord_encoded']

ord_encoded_df = pd.concat([df, ord_encoded], axis = 1)

ord_encoded

## Frequency Encoding


In [ ]:
frequency_encoded = df.groupby(['neighbourhood']).size()

frequency_encoded

# replace the categories with these different frequencies
df['neighbourhood'].apply(lambda x: frequency_encoded[x])

In [ ]:
class FrequencyEncoder:
      """
    A simple frequency encoder for categorical variables.

    This encoder replaces categorical values with their frequency counts based
    on the distribution in the training dataset.
    """
    def fit(self, train_df, column):
        """
        Fits the encoder by storing the reference DataFrame and column.

        Parameters:
        ----------
        train_df : pandas.DataFrame
            The training DataFrame containing the categorical column.
        column : str
            The name of the categorical column to encode.
        """
        self.train_df = train_df
        self.column = column

    def transform(self, test_df, column):
        """
        Transforms the test DataFrame by replacing categorical values with
        their frequency counts from the training set.

        Parameters:
        ----------
        test_df : pandas.DataFrame
            The DataFrame to be transformed.
        column : str
            The categorical column to encode.

        Returns:
        -------
        pandas.DataFrame
            The transformed DataFrame with an added frequency-encoded column.
        """
        frequency_encoded = self.train_df.groupby([self.column]).size()

        col_name = column + '_freq'
        test_df.loc[:,col_name] = test_df[column].apply(lambda x: frequency_encoded[x])
        return test_df

# frequency_encoding(df, column='neighbourhood')

fe = FrequencyEncoder()
fe.fit(df, column='neighbourhood')
df_freq_enc = fe.transform(df, column='neighbourhood')

df_freq_enc['neighbourhood_freq']

## Target Encoding  

 Rather than encode the number of instances a category occurs, we can encode the mean of our target variable, like this:

In [ ]:
df.groupby(['neighbourhood']).mean()['price']

In [ ]:
class TargetEncoder:
    def fit(self, train_df, target_col, categ_col):
        self.train_df = train_df
        self.target_col = target_col
        self.categ_col = categ_col

    def transform(self, test_df, column = None):
        if column is None:
            column = self.categ_col

        target_encoder = self.train_df.groupby([self.categ_col]).mean()[self.target_col]

        df[self.categ_col].apply(lambda x: target_encoder[x])

        col_name = column + '_target_enc'
        test_df.loc[:,col_name] = test_df[column].apply(lambda x: target_encoder[x])
        return test_df

te = TargetEncoder()
te.fit(df, target_col = 'price', categ_col = 'neighbourhood')

te_df = te.transform(df)

te_df

## Probability Ratio Encoding  

Probability Ratio Encoding is similar to target encoding. But rather than using the mean of the target, we're looking at the probability this category is going to be a positive label.


In [ ]:
class ProbabilityRatioEncoder:
    def fit(self, train_df, categ_col, target_col):
        self.train_df = train_df
        self.categ_col = categ_col
        self.target_col = target_col

    def transform(self, test_df, constant = 0):
        totals = self.train_df.groupby([self.categ_col]).size()
        sums = self.train_df.groupby([self.categ_col]).sum()[self.target_col]

        ratio_encoder = (sums+ constant)/totals

        col_name = self.categ_col + '_prob_ratio'
        test_df.loc[:,col_name] = test_df[self.categ_col].apply(lambda x: ratio_encoder[x])
        return test_df

# Example
pre = ProbabilityRatioEncoder()

pre.fit(df, 'neighbourhood','expensive')
pre_df = pre.transform(df)

pre_df

## Weight of Evidence Encoder  

Weight of Evidence encoding is similar to probability ratio encoding. The only difference, is we're applying a log transform on top of the probability ratio transformation:


In [ ]:
class WeightofEvidenceEncoder:
    def fit(self, train_df, categ_col, target_col):
        self.train_df = train_df
        self.categ_col = categ_col
        self.target_col = target_col

    def transform(self, test_df, constant = 0):
        totals = self.train_df.groupby([self.categ_col]).size()
        sums = self.train_df.groupby([self.categ_col]).sum()[self.target_col]

        woe_encoder = np.log((sums+ constant)/totals)

        col_name = self.categ_col + '_woe'
        test_df.loc[:,col_name] = test_df[self.categ_col].apply(lambda x: woe_encoder[x])
        return test_df

woe = WeightofEvidenceEncoder()

woe.fit(df, 'neighbourhood','expensive')
woe_df = woe.transform(df)

woe_df

## Binning  

The last technique is called binning. This is where we take a continuous variable and bin them into different buckets, thus, transforming this continuous variable into a categorical variable


In [ ]:
df.loc[:,'last_review'] = pd.to_datetime(df['last_review'])

def reviews_bin(x):
    if x < 50:
        return 'less_50'
    if x >= 50 and x < 100:
        return '50_to_100'
    if x >= 100 and x < 150:
        return '100_to_150'
    if x >= 150 and x < 200:
        return '150_to_200'
    if x >= 200:
        return '200_plus'

df['number_of_reviews'].map(reviews_bin)

# Continuous Features


## Scaling  

Feature scaling is important for we are using models with a distance metric. If our features are of different scales, they can be overcompensated for in the models.


### Absolute Max Scaling


In [ ]:
# Prepare the data
from sklearn.model_selection import train_test_split

X = df_example.drop('price',axis =1 )
y = df_example[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

#Scale data
df_am = MaxAbsScaler().fit_transform(X_train)

#convert to dataframe to see table
df_am = pd.DataFrame(df_am, columns = X_train.columns)

#obvious problems with outliers regarding price & odometer

### MinMax Scaling  

Between [0 and 1]



In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Be carefule of the data leakage
df_min_max = MinMaxScaler().fit_transform(X_train)
df_min_max = pd.DataFrame(df_min_max, columns = X_train.columns)


# fit and transform in different steps

# load the packages
from sklearn.preprocessing import MinMaxScaler

# Step 1: Create a dataset
data = np.array([[10], [20], [30], [40], [50]], dtype=float)

# Step 2: Initialize MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))  # Scale between 0 and 1

# Step 3: Fit the scaler to the data (calculates min and max)
scaler.fit(data)

# Step 4: Transform the data (scales using min-max formula)
scaled_data = scaler.transform(data)

# Step 5: Print results
print("Original Data:\n", data)
print("Scaled Data:\n", scaled_data)

# Checking the min_ and scale_ attributes
print("Data Min:", scaler.data_min_)
print("Data Max:", scaler.data_max_)
print("Scaling Factor:", scaler.scale_)

### Z-Score Normalization (Standard Scaler)


In [ ]:
from sklearn.preprocessing import StandardScaler
df_std = X_train.copy()
#only scale numeric varaibles in this case rather than the dummy variables for categories
df_std.loc[:,['car_age','odometer']] = StandardScaler().fit_transform(df_std.loc[:, ['car_age','odometer']])
df_std

### Robust Scaler


In [ ]:
from sklearn.preprocessing import RobustScaler
df_rob = X_train.copy()
#only scale numeric varaibles in this case rather than the dummy variables for categories
df_rob.loc[:,['car_age','odometer']] = RobustScaler().fit_transform(df_rob.loc[:, ['car_age','odometer']])
df_rob

## Transformations  

A data transformation is the process of using a math expression to change the structure of our data. As we mentioned before, some models need data to fit a specific type of distribution for them to produce optimal results. Unfortunately, the data we get in the real world, doesn’t always fit the distributions our models call for.


### Logarithmic


In [ ]:
from sklearn.preprocessing import FunctionTransformer

def log_transform(x):
    return np.log(x + 1)

transformer_log = FunctionTransformer(log_transform)
transformed_log = transformer_log.fit_transform(X_train)

In [ ]:
transformer_logp = FunctionTransformer(log_transform)
transformed_logp = transformer_logp.fit_transform(y_train)

### Square Root


In [ ]:
def sqrt_transform(x):
    return np.sqrt(x)

transformer_sqrt = FunctionTransformer(sqrt_transform)
transformed_sqrt = transformer_sqrt.fit_transform(X_train)

In [ ]:
transformer_sqrtp = FunctionTransformer(sqrt_transform)
transformed_sqrtp = transformer_sqrtp.fit_transform(y_train)

### Exponential


In [ ]:
def exp_transform(x):
    return np.exp(x)

transformer_exp = FunctionTransformer(exp_transform)

## In our dataset, car age may be something we want to magnify
transformed_exp = X_train.copy()

transformed_exp['car_age'] = transformer_exp.fit_transform(transformed_exp['car_age'])

### Box-Cox  

The Box-Cox transformation is a transformation that helps your dataset follow a normal distribution. Typically, we use Box-Cox transformation when our dataset is not normal, but close to being normal. When we want to either run tests or generate significance from our dataset, Box-Cox transformation is a good option to transform our target variable so it resembles a normal distribution.

Box-Cox aggregates multiple power transformers into a single transformer. You use lambda to adjust the transformation. Lambda varies from -5 to 5. If we set lambda equal to zero, it becomes simply a log transformation.



In [ ]:
## Redo the pipeline for this example
from sklearn.model_selection import train_test_split

## Clip Outliers
df_example = df_example[df_example['price'] < np.percentile(df_example['price'], 95)]

## Remove prices that are 0 to make notebook work
df_example = df_example[df_example['price'] > 0].copy()

X = df_example.drop('price',axis =1 )
y = df_example[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from scipy.stats import boxcox

boxcox_y_train = boxcox(y_train['price'], lmbda = None)

plt.hist(boxcox_y_train[0])

In [ ]:
print("Lambda Parameter {0}".format(boxcox_y_train[1]))

## Interaction Features  

While the interaction could be with the variable itself, we can also take differences, ratios and mutliples of two or more variables.


### Arethmetic Interaction


In [ ]:
#first example of getting car's age:
df['car_age'] = df['year'].max() - df['year']

#let's look at price per mile. This could be a good way to normalize across different car brands
df['price_per_mile'] = df['price']/ df['odometer']

### Binning


In [ ]:
#create warranty bin > 50,000 miles
# we are using a lambda function here. This lets us write a function without defining it
# we are also using a ternary operator which is an if, else statement in a single line. (explained in the full video)

df['warranty_miles'] = df['odometer'].apply(lambda x: 0 if (x > 50000 or np.isnan(x)) else 1)
df['warranty_age'] = df['car_age'].apply(lambda x: 0 if (x > 5 or np.isnan(x)) else 1)


# We can also combine these together in a single statement by defining a function.
def warranty(miles, age):
    if (miles > 50000 or age > 5):
        return 0
    else:
        return 1

df['warranty'] = df.apply(lambda x: min(x.warranty_miles,x.warranty_age), axis=1)


# Dealing with Imbalanced Data

## Diagnosis

In [ ]:
df_train.target.value_counts()

In [ ]:
# see imblance in data majority is not looking for a job change (0)
y_train.value_counts().plot.bar()

## Random Oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler

o_smpl = RandomOverSampler(random_state = 42)

X_o_smpl, y_o_smpl = o_smpl.fit_resample(X_train,y_train)

y_o_smpl.value_counts().plot.bar()

## Random Undersampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

u_smpl = RandomUnderSampler(random_state = 42)

X_u_smpl, y_u_smpl = u_smpl.fit_resample(X_train,y_train)

In [ ]:
#plot
y_u_smpl.value_counts().plot.bar()

## Synthetic Minority Oversampling (SMOTE)

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state = 42)

X_smote, y_smote = smote.fit_resample(X_train,y_train)

#plot
y_smote.value_counts().plot.bar()

## Borderline SMOTE

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE

bsmote = BorderlineSMOTE(random_state = 42)

X_bsmote, y_bsmote = bsmote.fit_resample(X_train,y_train)

#plot
y_smote.value_counts().plot.bar()

## Adaptive Synthetic Oversampling (ADASYN)

In [ ]:
from imblearn.over_sampling import ADASYN
adasyn = ADASYN(random_state = 42)

X_ada, y_ada = adasyn.fit_resample(X_train,y_train)

# plot
y_ada.value_counts().plot.bar()


# Merge datasets

## Merging datasets on a common key

In [ ]:
# Merging datasets on a common key 'customer_id'
merged_data = pd.merge(data1, data2, on='customer_id', how='inner')

## Joining on column(s) with the same name

In [ ]:
join1 = pd.merge(
    left=df1,  # Specify DataFrame on the left to merge
    right=df2,  # Specify DataFrame on the right to merge
    how="left",  # Choose 'inner', 'outer', 'cross' 'left' or 'right'
    on=["uid"],  # List of column(s) to merge on (these must exist in both DataFrames)
    indicator=True,  # When true, adds “_merge” column with source of each row
)

print("Number of rows & columns:", join1.shape)
join1.head()

## Joining on column(s) with different names

In [ ]:
join2 = pd.merge(
    left=df1,  # Specify DataFrame on the left to merge
    right=df2,  # Specify DataFrame on the right to merge
    how="inner",  # Choose 'inner', 'outer', 'cross' 'left' or 'right'
    left_on=["reg_date", "uid"],  # List of column(s) to merge on in the left DataFrame
    right_on=["date", "uid"],  # List of column(s) to merge on in the right DataFrame
    indicator=True,  # When true, adds “_merge” column with source of each row
)

print("Number of rows & columns:", join2.shape)
join2.head()

# Scaling numeric data

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Creating a manual dataset
data = pd.DataFrame({
    'category': ['A', 'B', 'A', 'C', 'B'],
    'numeric_column': [10, 15, 10, 20, 15]
	})

# Scaling numeric data
scaler = StandardScaler()
data['scaled_numeric_column'] = scaler.fit_transform(data[['numeric_column']])

## one-hot encoding

In [ ]:
# Creating a manual dataset
data = pd.DataFrame({
    'category': ['A', 'B', 'A', 'C', 'B'],
    'numeric_column': [10, 15, 10, 20, 15]
	})

# Encoding categorical variables using one-hot encoding
encoder = OneHotEncoder(sparse_output=False)
encoded_data = pd.DataFrame(encoder.fit_transform(data[['category']]),
                            columns=encoder.get_feature_names_out(['category']))

# Concatenating the encoded data with the original dataset
data = pd.concat([data, encoded_data], axis=1)

## Features selection

In [ ]:
features = []
for column in df.columns:
    if column != 'output':        # Change 'output' with the output name variable
        features.append(column)
X = df[features]
Y = df['output']                   # Change 'output' with the output name variable

## Visualize correlation

In [ ]:
# Load packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid")
%config InlineBackend.figure_format = 'retina'

# Upload your data as CSV and load as data frame
df = pd.read_csv('mpg.csv')
df

# Compute the correlation matrix
corr = df.corr(method = 'pearson')

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(11, 9))                    # Set figure size

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask
sns.heatmap(corr,
            mask = mask,
            cmap = cmap,
            vmax = 1,                                      # Set scale min value
            vmin = -1,                                     # Set scale min value
            center = 0,                                    # Set scale min value
            square = True,                                 # Ensure perfect squares
            linewidths = 1.5,                              # Set linewidth between squares
            cbar_kws = {"shrink": .9},                     # Set size of color bar
            annot = True                                   # Include values within squares
           );

plt.xticks(rotation=45)                                    # Rotate x labels
plt.yticks(rotation=45)                                    # Rotate y labels
# plt.xlabel('X Axis Title', size=20)                      # Set x axis title
# plt.ylabel('Y Axis Title', size=20)                      # Set y axis title
plt.title('Diagonal Correlation Plot', size=30, y=1.05);   # Set plot title and position

## Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

for column in X.columns:
    feature = np.array(X[column]).reshape(-1,1)
    scaler = MinMaxScaler()
    scaler.fit(feature)
    feature_scaled = scaler.transform(feature)
    X[column] = feature_scaled.reshape(1,-1)[0]